In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rcParams
import scipy.constants as sc
import pandas as pd
rcParams["figure.figsize"] = (5,2)

In [3]:
r_in = 1 # in au
r_out = 300 # in au
P = 100 # number of R intervals
R = np.logspace(np.log10(r_in), np.log10(r_out), P)

c_s = 1.5 * R**(-0.25) # isothermal sound speed in km/s
Omega = (np.sqrt(sc.G * 2e30) * 1.5e11**(-1.5)) * R**(-1.5) # Keplerian freq in /s
sd = 1700 * R**(-1.5) # surface density in g/cm2
rhos = 4 # solid grain density in g/cm3
sdot = 2e-12 * (R/10)**(-1.5) # surface mass loss rate (Picogna+19) in g/cm2/s

H = c_s / Omega * 1e5 # scale height in cm
H_R = c_s / Omega / (1.5e8) / R
eta = H_R**2
u0 = np.sqrt(2*np.pi) / Omega * sdot / sd # midplane gas velocity
rho0 = sd / (np.sqrt(2*np.pi) * H) # midplane gas density

b = sdot / (Omega * rhos) # cst I defined that scales 1/a

In [3]:
# scale height at 100 au
H0 = 1.5 * 100**(-0.25) / ((np.sqrt(sc.G * 2e30) * 1.5e11**(-1.5)) * 100**(-1.5)) / 1.5e8
print('Scale height at 100 au:', round(H0,1), 'au') # need this to update param file

# max settling height - just for information
msh = 1/np.sqrt(2*eta)

Scale height at 100 au: 15.9 au


In [10]:
eps = 1e-4 # minimum height above midplane considered in H
N = 70 # number of z intervals
Z = np.linspace(0,10,N)

# generate lookup table for grain sizes in each cell
a_limits = b * ( 1 + eta * eps**2 )**(1.5) / eps
a_limits = np.reshape(a_limits, (1,len(a_limits)))
for i in Z[1:]:
    a_lim_append = b * ( 1 + eta * i**2 )**(1.5) / i
    a_limits = np.vstack((a_limits, np.reshape(a_lim_append, (1,len(a_lim_append)))))

s=0
for i in np.argmin(a_limits, axis=0):
    a_limits[i+1:,s] =  1e30 # fill value
    s+=1
del s

# generate grain size list (log dist)
M = 100 # number of grain sizes
a_min = 0.03 / 1e4 # in cm
a_max = 1000 / 1e4 # in cm
print('Grain size range: [',a_min,',',a_max,'] cm')
A = np.logspace(np.log10(a_min), np.log10(a_max), M)

Grain size range: [ 3e-06 , 0.1 ] cm


In [5]:
# if grain size is smaller than the minimum in this array, then it escapes and doesn't have a density
# I am limited by 10H being the maximum height but at radii smaller than 4 au, the maximum settling
# height is greater than 10H. can get around this by enforcing the inner radius of the disc to be > 4 au. 
R[np.argmin(np.abs(msh-10))]

3.985788443006654

In [6]:
pd.DataFrame(data=a_limits)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.795040,0.795040,0.795040,0.795040,0.795040,0.795040,0.795040,0.795040,0.795040,0.795040,...,7.950401e-01,7.950401e-01,7.950401e-01,7.950401e-01,7.950401e-01,7.950401e-01,7.950401e-01,7.950401e-01,7.950401e-01,7.950401e-01
1,0.000549,0.000549,0.000549,0.000549,0.000549,0.000549,0.000549,0.000549,0.000549,0.000549,...,5.491618e-04,5.491789e-04,5.491964e-04,5.492145e-04,5.492332e-04,5.492523e-04,5.492720e-04,5.492923e-04,5.493132e-04,5.493347e-04
2,0.000274,0.000274,0.000274,0.000274,0.000274,0.000274,0.000274,0.000274,0.000274,0.000274,...,2.754577e-04,2.754919e-04,2.755271e-04,2.755633e-04,2.756006e-04,2.756390e-04,2.756784e-04,2.757191e-04,2.757609e-04,2.758040e-04
3,0.000183,0.000183,0.000183,0.000183,0.000183,0.000183,0.000183,0.000183,0.000183,0.000183,...,1.846141e-04,1.846655e-04,1.847184e-04,1.847728e-04,1.848288e-04,1.848865e-04,1.849458e-04,1.850069e-04,1.850698e-04,1.851346e-04
4,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,...,1.394871e-04,1.395558e-04,1.396265e-04,1.396993e-04,1.397742e-04,1.398513e-04,1.399307e-04,1.400124e-04,1.400965e-04,1.401831e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.000011,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30
66,0.000011,0.000011,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30
67,0.000011,0.000011,0.000011,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30
68,0.000011,0.000011,0.000011,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,...,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30


In [4]:
# need to calculate power law scaling for each radius???
rho0

array([8.99184267e-10, 7.67431772e-10, 6.54984242e-10, 5.59013026e-10,
       4.77103941e-10, 4.07196541e-10, 3.47532286e-10, 2.96610303e-10,
       2.53149637e-10, 2.16057022e-10, 1.84399382e-10, 1.57380361e-10,
       1.34320288e-10, 1.14639080e-10, 9.78416497e-11, 8.35054543e-11,
       7.12698622e-11, 6.08270838e-11, 5.19144278e-11, 4.43076940e-11,
       3.78155329e-11, 3.22746321e-11, 2.75456089e-11, 2.35095033e-11,
       2.00647859e-11, 1.71248038e-11, 1.46156011e-11, 1.24740580e-11,
       1.06463033e-11, 9.08635940e-12, 7.75498544e-12, 6.61869035e-12,
       5.64889028e-12, 4.82118965e-12, 4.11476742e-12, 3.51185332e-12,
       2.99728089e-12, 2.55810591e-12, 2.18328081e-12, 1.86337675e-12,
       1.59034647e-12, 1.35732180e-12, 1.15844095e-12, 9.88701011e-13,
       8.43832124e-13, 7.20190073e-13, 6.14664607e-13, 5.24601204e-13,
       4.47734293e-13, 3.82130265e-13, 3.26138832e-13, 2.78351514e-13,
       2.37566207e-13, 2.02756946e-13, 1.73048093e-13, 1.47692313e-13,
      

In [14]:
H

array([7.54241263e+11, 8.10563552e+11, 8.71091659e+11, 9.36139647e+11,
       1.00604503e+12, 1.08117055e+12, 1.16190599e+12, 1.24867027e+12,
       1.34191361e+12, 1.44211980e+12, 1.54980880e+12, 1.66553938e+12,
       1.78991204e+12, 1.92357212e+12, 2.06721314e+12, 2.22158043e+12,
       2.38747495e+12, 2.56575750e+12, 2.75735314e+12, 2.96325602e+12,
       3.18453450e+12, 3.42233676e+12, 3.67789668e+12, 3.95254032e+12,
       4.24769272e+12, 4.56488537e+12, 4.90576409e+12, 5.27209762e+12,
       5.66578678e+12, 6.08887432e+12, 6.54355555e+12, 7.03218969e+12,
       7.55731215e+12, 8.12164765e+12, 8.72812440e+12, 9.37988926e+12,
       1.00803241e+13, 1.08330632e+13, 1.16420125e+13, 1.25113694e+13,
       1.34456446e+13, 1.44496861e+13, 1.55287034e+13, 1.66882954e+13,
       1.79344789e+13, 1.92737200e+13, 2.07129678e+13, 2.22596901e+13,
       2.39219125e+13, 2.57082598e+13, 2.76280011e+13, 2.96910972e+13,
       3.19082533e+13, 3.42909735e+13, 3.68516212e+13, 3.96034829e+13,
      

In [19]:
np.full((100,),(np.roll(Z,-1) - Z)[:-1])

ValueError: could not broadcast input array from shape (69,) into shape (100,)

In [52]:
delta_r2 = 4*np.pi * ((np.roll(R,-1))**2 - R**2) * (1.5e13)**2 # area of radial bin in cm2
delta_z = np.multiply(np.full((N,P), (np.roll(Z,-1) - Z)[0]), H.T ) # size of vertical bin in cm
delta_vol = delta_r2 * delta_z # in cm3

print(delta_vol.shape)

scale = rho0 / 100 * 3 / (8*np.pi) / rhos / (np.sqrt(a_max) - np.sqrt(a_min))

(70, 100)


In [56]:
scale

array([8.53208886e-13, 7.28192910e-13, 6.21494833e-13, 5.30430635e-13,
       4.52709570e-13, 3.86376542e-13, 3.29762926e-13, 2.81444589e-13,
       2.40206071e-13, 2.05010005e-13, 1.74971024e-13, 1.49333488e-13,
       1.27452478e-13, 1.08777572e-13, 9.28389965e-14, 7.92358122e-14,
       6.76258272e-14, 5.77169890e-14, 4.92600380e-14, 4.20422373e-14,
       3.58820209e-14, 3.06244270e-14, 2.61371992e-14, 2.23074601e-14,
       1.90388714e-14, 1.62492109e-14, 1.38683040e-14, 1.18362582e-14,
       1.01019568e-14, 8.62177295e-15, 7.35847227e-15, 6.28027606e-15,
       5.36006196e-15, 4.57468175e-15, 3.90437895e-15, 3.33229191e-15,
       2.84402962e-15, 2.42730970e-15, 2.07164944e-15, 1.76810212e-15,
       1.50903190e-15, 1.28792181e-15, 1.09920975e-15, 9.38148631e-16,
       8.00686904e-16, 6.83366684e-16, 5.83236746e-16, 4.97778293e-16,
       4.24841595e-16, 3.62591907e-16, 3.09463322e-16, 2.64119374e-16,
       2.25419424e-16, 1.92389963e-16, 1.64200127e-16, 1.40140791e-16,
      

In [41]:
# start with one grain size and then generate Z/R grid
n_A = []

for j in A:
    a_loc = j
    # print(a_loc)
    q = ~(a_loc < a_limits)
    r = np.sum(q, axis=0)

    n_ZR = np.zeros((N,P))
    if np.sum(r)>0:
        for i in np.where(r!=0)[0]:
            n_ZR[N-r[i],i] = scale[i] * a_loc**(-3.5) * delta_vol[N-r[i],i] # NEED TO CHANGE THIS...

    n_A_append = np.reshape(n_ZR, (1, N, P))
    n_A.append(n_A_append)

n_A = np.array(n_A)

In [51]:
n_A[31][0][:, 4]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       8.53403597e+39, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      